# Agents

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
from langchain_google_genai import GoogleGenerativeAI
import google.generativeai as genai

gemini_api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key = gemini_api_key)

In [5]:
llm = GoogleGenerativeAI(model = "gemini-1.5-flash",temperature=0.7)

response = llm.invoke("What is called langchain")
response

"LangChain is a framework for developing applications powered by large language models (LLMs).  It simplifies the process of building LLM-based applications by providing tools and components for:\n\n* **Connecting to various LLMs:**  It allows you to easily switch between different LLMs (like OpenAI's GPT models, Hugging Face models, etc.) without modifying your core application code.\n\n* **Managing interactions with LLMs:** It handles prompt engineering, chaining multiple calls to LLMs, and managing the context of the conversation.\n\n* **Integrating with other data sources:**  It allows you to connect LLMs to your own databases, documents, APIs, and other data sources to provide them with more context and capabilities.\n\n* **Building different application types:** LangChain provides building blocks for various applications such as chatbots, question-answering systems, summarization tools, and more.\n\nIn essence, LangChain acts as an abstraction layer, making it easier to work with

In [6]:
from langchain.tools import Tool
from langchain.agents import initialize_agent
from langchain.chains import LLMMathChain

llm_math = LLMMathChain(llm=llm)

math_tool = Tool(
    name= "calculator",
    func = llm_math.run ,
    description= "Useful for when you need to answer questions about math."
)

tools = [math_tool]

tools[0].name, tools[0].description

C:\Users\Parthiban\AppData\Local\Temp\ipykernel_13696\3364858757.py:5: LangChainDeprecationWarning: This class is deprecated and will be removed in langchain 1.0. See API reference for replacement: https://api.python.langchain.com/en/latest/chains/langchain.chains.llm_math.base.LLMMathChain.html
  llm_math = LLMMathChain(llm=llm)
d:\Users\Lib\site-packages\langchain\chains\llm_math\base.py:175: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


('calculator', 'Useful for when you need to answer questions about math.')

In [7]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm = llm
)
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [8]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

C:\Users\Parthiban\AppData\Local\Temp\ipykernel_13696\298812146.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  zero_shot_agent = initialize_agent(


In [9]:
zero_shot_agent("what is (4.5*2.1)^2.2?")

C:\Users\Parthiban\AppData\Local\Temp\ipykernel_13696\2247149980.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  zero_shot_agent("what is (4.5*2.1)^2.2?")




> Entering new AgentExecutor chain...
Thought: I need to calculate (4.5 * 2.1) raised to the power of 2.2.  I can use the calculator for this.
Action: Calculator
Action Input: (4.5 * 2.1)**2.2
Observation: Answer: 139.94261298333066
Thought:Thought: I now know the final answer
Final Answer: 139.94261298333066


> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

In [10]:
(4.5*2.1)**2.2

139.94261298333066

In [11]:
zero_shot_agent("if Mary has four apples and Giorgio brings two and a half apple "
                "boxes (apple box contains eight apples), how many apples do we "
                "have?")



> Entering new AgentExecutor chain...
Thought: I need to calculate the total number of apples.  First, I need to find the total number of apples in the apple boxes, and then add that to the number of apples Mary has.

Action: Calculator
Action Input: 2.5 * 8

Observation: Answer: 20.0
Thought:Thought: Now I need to add Mary's apples to the total from the apple boxes.

Action: Calculator
Action Input: 20 + 4

Observation: Answer: 24
Thought:Thought: I now know the final answer
Final Answer: 24


> Finished chain.


{'input': 'if Mary has four apples and Giorgio brings two and a half apple boxes (apple box contains eight apples), how many apples do we have?',
 'output': '24'}

In [ ]:
zero_shot_agent("what is the capital of Norway?") # No need for tool



> Entering new AgentExecutor chain...
Thought: I don't need a calculator to answer this question.  I need to access general knowledge.
Thought: I know the capital of Norway is Oslo.
Final Answer: Oslo


> Finished chain.


{'input': 'what is the capital of Norway?', 'output': 'Oslo'}